In [ ]:
import sys
import os
from crewai import Process, Crew, Agent, Task
from crewai_tools import BaseTool, SerperDevTool
import requests
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [ ]:
load_dotenv()
search_tool = SerperDevTool()

1. Knowledge Retrieval Agent

In [ ]:
knowledge_agent = Agent(
    role="AWS Documentation Specialist",
    goal="Retrieve and synthesize comprehensive AWS documentation tailored specifically to the user's requirements and use case",
    backstory=(
        "You are an AWS certified Solutions Architect with 10+ years of experience specializing in AWS documentation. "
        "You have an encyclopedic knowledge of AWS services, their limitations, integration patterns, and best practices. "
        "Your primary responsibility is to extract actionable insights from AWS documentation, white papers, implementation guides, "
        "and reference architectures that specifically address the user's use case requirements. "
        "You understand the subtle technical distinctions between similar AWS services and which are optimal for specific scenarios. "
        "You're not just a documentation retriever but a documentation curator who knows what information is most critical "
        "for implementation success in the specific domain of the user's needs."
    ),
    allow_delegation=False,
    verbose=True,
)

2. Industry Solutions Expert Agent 

In [ ]:
industry_solutions_agent = Agent(
    role="Industry Solutions Expert",
    goal="Identify industry-specific AWS patterns and real-world case studies that align with the user's use case",
    backstory=(
        "You are an AWS Technical Account Manager with experience across multiple industries, specializing in translating "
        "business requirements into technical solutions. Your expertise lies in identifying proven AWS implementation patterns "
        "for specific use cases from AWS's library of case studies, partner solutions, and the AWS Solutions Library. "
        "You focus on extracting the detailed architectural components from these solutions and understanding how they were "
        "implemented to meet specific industry challenges. Rather than just naming services, you provide insight into the "
        "exact configuration, integration patterns, and technical decisions that made these solutions successful."
    ),
    allow_delegation=True,
    verbose=True,
)

3. Implementation Details Agent 

In [ ]:
implementation_details_agent = Agent(
    role="Implementation Architect",
    goal="Convert high-level architecture components into detailed implementation specifications with code and configuration examples",
    backstory=(
        "You are a hands-on AWS implementation specialist who transforms architectural diagrams into executable code. "
        "Your expertise is in Infrastructure as Code (CloudFormation, Terraform, CDK), AWS CLI commands, and AWS SDK implementations. "
        "You specialize in searching GitHub repositories and AWS documentation for specific implementation patterns that match "
        "the proposed architecture. Rather than simply finding examples, you adapt and enhance them to fit the user's specific requirements. "
        "You provide explicit step-by-step implementation instructions, including actual code snippets, configuration files, and command sequences "
        "needed to deploy the architecture. Your recommendations include details like IAM policies, network configurations, security groups, "
        "and service-specific parameter optimizations that are essential for implementation."
    ),
    allow_delegation=True,
    verbose=True,
)

4. Technical Constraints Specialist Agent

In [ ]:
constraints_agent = Agent(
    role="Technical Constraints Specialist",
    goal="Identify and document implementation limitations, quotas, integration challenges and their solutions",
    backstory=(
        "You are an AWS technical specialist who focuses exclusively on the practical limitations and edge cases "
        "that impact real-world implementations. You have deep knowledge of AWS service quotas, integration constraints, "
        "and known implementation challenges. You specialize in identifying potential bottlenecks, limitations, and technical "
        "constraints that could affect the architecture, and providing specific workarounds or solutions for each issue. "
        "Your expertise covers performance limitations, service quotas, API throttling concerns, cross-service integration challenges, "
        "IAM permission boundaries, and region-specific limitations that are critical to implementation success but often "
        "overlooked in high-level architecture documentation."
    ),
    allow_delegation=True,
    verbose=True,
)

5. QA and Validation Agent

In [ ]:
qa_validation_agent = Agent(
    role="Solution Validation Expert",
    goal="Critically evaluate the proposed architecture against real-world implementation requirements and AWS best practices",
    backstory=(
        "You are a senior AWS consultant who specializes in pre-implementation validation and quality assurance. "
        "Your responsibility is to identify gaps, contradictions, or suboptimal choices in proposed AWS architectures. "
        "You leverage your experience from hundreds of AWS implementations to evaluate solutions against AWS's Well-Architected Framework, "
        "but more importantly, against practical implementation considerations. For every component in the architecture, you verify that it "
        "is the optimal choice given the specific requirements, budget constraints, and technical specifications. "
        "You're not satisfied with generic 'best practices' - you provide specific, actionable feedback on how to improve "
        "the architecture for real-world implementation, including alternative approaches that may better meet the requirements."
    ),
    allow_delegation=True,
    verbose=True,
)

6. Final Architecture Synthesizer Agent

In [ ]:
architecture_synthesizer = Agent(
    role="Lead Solutions Architect",
    goal="Create a comprehensive, implementation-ready AWS architecture document with detailed deployment instructions",
    backstory=(
        "You are a senior AWS Solutions Architect with 15+ years of experience leading complex cloud implementations. "
        "Your specialty is synthesizing inputs from multiple technical domains into coherent, executable implementation plans. "
        "You understand that valuable architecture documentation must go beyond service selection to include comprehensive "
        "deployment guidance, configuration details, and implementation steps. Your documentation includes detailed component "
        "interactions, data flows, scaling mechanisms, security implementations, and operational considerations. "
        "You bring a consultant's perspective to solution documentation - anticipating implementation challenges, providing "
        "specific configuration guidance, and explaining architectural decisions with their trade-offs. Your solutions include "
        "precise implementation steps that address the specific requirements of the use case with concrete examples and references."
    ),
    allow_delegation=False,
    verbose=True,
)

Task 1: Knowledge Retrieval and Synthesis

In [ ]:
task_knowledge_retrieval = Task(
    description=(
        "Extract and synthesize AWS documentation specifically related to the user's use case. "
        "Focus especially on the 'use_case' field in the requirements to guide your search. "
        "Retrieve information about AWS services, best practices, reference architectures, and implementation guides "
        "that specifically address this use case. "
        "For each recommended service or component:\n"
        "1. Document its specific purpose in the architecture\n"
        "2. Detail integration points with other services\n"
        "3. Identify configuration options specifically relevant to the use case\n"
        "4. Note any service limitations or quotas that might impact implementation\n"
        "Your output should be structured as a comprehensive knowledge base that can be directly used to create the implementation architecture."
    ),
    expected_output=(
        "A structured knowledge base document that includes:\n"
        "- Core AWS services recommended for the use case with specific justifications\n"
        "- Detailed integration patterns between services\n"
        "- Configuration parameters that need to be set for each service\n"
        "- Implementation considerations specific to the requirements\n"
        "- Direct references to AWS documentation and guides"
    ),
    agent=knowledge_agent,
    tools=[search_tool],
)

Task 2: Industry Solutions Analysis

In [ ]:
task_industry_solutions = Task(
    description=(
        "Research and analyze AWS reference architectures, case studies, and partner solutions that specifically address "
        "the user's use case. Focus on extracting the specific technical implementation details rather than just high-level patterns. "
        "For each relevant case study or reference architecture:\n"
        "1. Document the exact AWS services used and their specific configuration\n"
        "2. Detail how the solution addressed requirements similar to the user's (cost, scalability, security, etc.)\n"
        "3. Identify specific implementation decisions that were critical to the solution's success\n"
        "4. Note any lessons learned or optimizations that were made post-implementation\n"
        "Your output should provide concrete, implementable patterns that have been proven in real-world scenarios."
    ),
    expected_output=(
        "A detailed analysis of industry solutions that includes:\n"
        "- Specific implementations of the use case from AWS case studies\n"
        "- Technical architecture details from these implementations\n"
        "- Configuration choices made in these implementations\n"
        "- How these implementations addressed requirements similar to the user's\n"
        "- Specific implementation tips extracted from these real-world examples"
    ),
    agent=industry_solutions_agent,
    tools=[search_tool],
)

Task 3: Implementation Details Specification

In [ ]:
task_implementation_details = Task(
    description=(
        "Develop detailed implementation specifications for each component of the architecture. "
        "Search GitHub repositories, AWS samples, and documentation for specific code examples and configuration files. "
        "For each major component in the architecture:\n"
        "1. Provide actual infrastructure-as-code examples (CloudFormation, Terraform, or CDK)\n"
        "2. Include specific AWS CLI commands or SDK code snippets for setup and configuration\n"
        "3. Detail IAM policies, security group configurations, and other security-related settings\n"
        "4. Provide network configuration details, including VPC setup, subnet configuration, and routing\n"
        "5. Include service-specific parameter configurations optimized for the use case\n"
        "Your output should be structured as step-by-step implementation instructions with actual code that could be used to deploy the solution."
    ),
    expected_output=(
        "A detailed implementation guide that includes:\n"
        "- Infrastructure-as-code templates for major components\n"
        "- AWS CLI commands and SDK code snippets\n"
        "- Complete IAM policy documents\n"
        "- Security group configurations\n"
        "- Network configuration details\n"
        "- Service parameter settings\n"
        "- Step-by-step deployment instructions"
    ),
    agent=implementation_details_agent,
    tools=[search_tool],
)

Task 4: Technical Constraints Analysis

In [ ]:
task_constraints_analysis = Task(
    description=(
        "Identify and document all technical constraints, limitations, and potential implementation challenges "
        "for the proposed architecture based on the user's requirements. For each component in the architecture:\n"
        "1. Document applicable service quotas and limits\n"
        "2. Identify potential performance bottlenecks and scaling limitations\n"
        "3. Detail integration challenges between services and their solutions\n"
        "4. Outline region-specific considerations based on availability requirements\n"
        "5. Document compliance-specific implementation requirements based on the user's compliance needs\n"
        "For each constraint or limitation identified, provide specific workarounds or solutions that can be implemented."
    ),
    expected_output=(
        "A comprehensive constraints analysis that includes:\n"
        "- Service quotas and limits for each component\n"
        "- Performance constraints and solutions\n"
        "- Integration challenges and their workarounds\n"
        "- Region-specific implementation details\n"
        "- Compliance-related technical considerations\n"
        "- Specific solutions and workarounds for each constraint"
    ),
    agent=constraints_agent,
    tools=[search_tool],
)

Task 5: Solution Validation and Enhancement

In [ ]:
task_validation = Task(
    description=(
        "Critically evaluate the proposed architecture against the user's specific requirements and AWS best practices. "
        "For each component and design decision in the architecture:\n"
        "1. Verify alignment with the user's stated requirements (cost profile, scalability, security tier, etc.)\n"
        "2. Evaluate against the AWS Well-Architected Framework principles\n"
        "3. Identify any suboptimal choices or missed opportunities for service selection\n"
        "4. Suggest specific improvements or alternatives that better meet the requirements\n"
        "5. Validate that implementation details are complete and technically sound\n"
        "Provide specific, actionable feedback on how to improve the architecture for real-world implementation."
    ),
    expected_output=(
        "A detailed validation report that includes:\n"
        "- Analysis of how well each component meets the user's requirements\n"
        "- Well-Architected Framework evaluation\n"
        "- Identification of suboptimal choices or missed opportunities\n"
        "- Specific architectural improvements with justifications\n"
        "- Implementation detail verification\n"
        "- Actionable enhancements to improve the architecture"
    ),
    agent=qa_validation_agent,
    tools=[search_tool],
)

Task 6: Final Architecture Synthesis

task_architecture_synthesis = Task(
    description=(
        "Create a comprehensive, implementation-ready AWS architecture document that synthesizes all previous findings "
        "into a cohesive solution tailored to the user's specific requirements. The document should include:\n"
        "1. Executive summary relating the architecture directly to the user's use case and requirements\n"
        "2. Detailed architecture diagram with all components and their interactions\n"
        "3. Component-by-component implementation specifications including:\n"
        "   a. Service configuration details\n"
        "   b. Integration specifications\n"
        "   c. Security implementations\n"
        "   d. Scaling mechanisms\n"
        "4. Step-by-step implementation plan with actual code examples and configuration files\n"
        "5. Implementation timeline and resource requirements\n"
        "6. Operational considerations and monitoring setup\n"
        "7. Cost estimation based on the user's usage patterns\n"
        "The document should be structured as a consultant's implementation guide that could be followed directly by the user."
    ),
    expected_output=(
        "A comprehensive AWS architecture implementation document that includes:\n"
        "- Executive summary\n"
        "- Detailed architecture diagram\n"
        "- Component-by-component specifications\n"
        "- Implementation code and configuration\n"
        "- Step-by-step deployment guide\n"
        "- Implementation timeline\n"
        "- Operational guidance\n"
        "- Cost estimation\n"
        "The document should be implementation-ready with sufficient detail for the user to deploy the solution."
    ),
    agent=architecture_synthesizer
)

In [ ]:
crew = Crew(
    agents=[
        knowledge_agent,
        industry_solutions_agent,
        implementation_details_agent,
        constraints_agent,
        qa_validation_agent,
        architecture_synthesizer
    ],
    tasks=[
        task_knowledge_retrieval,
        task_industry_solutions,
        task_implementation_details,
        task_constraints_analysis,
        task_validation,
        task_architecture_synthesis
    ],
    process=Process.sequential,  # Ensures logical step-by-step execution
    verbose=2,  # Set verbosity level for detailed logs during execution
)

In [ ]:
user_requirements = {
    'cost_profile': "Cost-Optimized",
    'scalability': "Medium",
    'implementation': {
        'ease': "Easy",
        'integration': "Moderate",
        'expertise': "Intermediate"
    },
    'use_case': "AI-powered customer support chatbot",
    'security_tier': "Enterprise",
    'performance': "High Performance",
    'availability': "High Availability (multi-region)",
    'compliance': ["GDPR", "HIPAA"],
    'implementation_time': "Short (days)"
}
crew.kickoff(inputs=user_requirements)